In [89]:
from SmartApi.smartConnect import SmartConnect
import pyotp
import pandas as pd
import requests
import pandas as pd
from datetime import datetime
from datetime import datetime, timedelta
import pandas_ta as ta


api_key = 'C46AImmE'
clientId = 'M507690'
pwd = '4663'
smartApi = SmartConnect(api_key)
token = "4ACNNSIZL6EV6UQ2ACSI23H5VQ"
totp=pyotp.TOTP(token).now()
correlation_id = "abc123"

# login api call

data = smartApi.generateSession(clientId, pwd, totp)
# print(data)
authToken = data['data']['jwtToken']
refreshToken = data['data']['refreshToken']

# fetch the feedtoken
feedToken = smartApi.getfeedToken()

# fetch User Profile
res = smartApi.getProfile(refreshToken)
smartApi.generateToken(refreshToken)
res=res['data']['exchanges']

#fetch User Profile
userProfile= smartApi.getProfile(refreshToken)
User= pd.DataFrame(userProfile)
User = User['data']['name']
User
print(f"Welcome {User} to the World of SMART API BY ANGEL.....!!")
print(f"Login Sucessfull....!!")


[I 251218 01:33:37 smartConnect:124] in pool


Welcome MANA  BHATTACHARYYA to the World of SMART API BY ANGEL.....!!
Login Sucessfull....!!


In [90]:
# ---- NIFTY 50 Token (Angel One Fixed Token) ----
NIFTY_TOKEN = "26000"      # NIFTY 50 Index
EXCHANGE = "NSE"

# ---- Fetch LTP ----
ltp_data = smartApi.ltpData(
    exchange=EXCHANGE,
    tradingsymbol="NIFTY",
    symboltoken=NIFTY_TOKEN
)

print("📈 NIFTY 50 LTP:", ltp_data['data']['ltp'])

ATM = round(ltp_data['data']['ltp'] / 50) * 50
print("📈 NIFTY 50 SPOT Price:", ATM)

ltp =  ltp_data['data']['ltp']
prev_close = ltp_data['data']['close']

change = ltp - prev_close
pct_change = (change / prev_close) * 100

print(f"NIFTY Change: {change:.2f}")
print(f"NIFTY % Change: {pct_change:.2f}%")


📈 NIFTY 50 LTP: 25818.55
📈 NIFTY 50 SPOT Price: 25800
NIFTY Change: -41.55
NIFTY % Change: -0.16%


In [91]:
import pandas as pd
import pandas_ta as ta

def add_indicators(df):
    df["VWAP"] = (df["Close"] * df["Volume"]).cumsum() / df["Volume"].cumsum()
    df["EMA14"] = ta.ema(df["Close"], length=20)
    df["RSI"] = ta.rsi(df["Close"], length=14)

    macd = ta.macd(df["Close"])
    df["MACD"] = macd["MACD_12_26_9"]
    df["MACD_SIGNAL"] = macd["MACDs_12_26_9"]

    return df

In [92]:
# ---------- FETCH 5-MIN CANDLES ---------- #
to_date = datetime.now()
from_date = to_date - timedelta(days=5)

params = {
    "exchange": "NSE",
    "symboltoken": "99926000",
    "interval": "FIVE_MINUTE",
    "fromdate": from_date.strftime("%Y-%m-%d %H:%M"),
    "todate": to_date.strftime("%Y-%m-%d %H:%M")
}

data = smartApi.getCandleData(params)
df = pd.DataFrame(
    data["data"],
    columns=["Datetime", "Open", "High", "Low", "Close", "Volume"]
)
df["Datetime"] = pd.to_datetime(df["Datetime"])
df = add_indicators(df)
df = df.tail(3)
df



,Datetime,Open,High,Low,Close,Volume,VWAP,EMA14,RSI,MACD,MACD_SIGNAL
222,2025-12-17 15:15:00+05:30,25817.15,25817.15,25809.2,25814.10,0,NaN,25806.685282,53.205245,3.602477,1.012421
223,2025-12-17 15:20:00+05:30,25813.65,25826.90,25811.5,25825.95,0,NaN,25808.520017,57.277285,4.630637,1.736064
224,2025-12-17 15:25:00+05:30,25825.15,25835.75,25821.4,25821.80,0,NaN,25809.784777,55.457218,5.052351,2.399322
